# 📊 4강: AI 피드백 시스템

> 학생/학부모 피드백 4종 자동 생성 + pandas 배치 처리로 5명 동시 생성

## ⚠️ 시작 전 필수! 노트북 복사하기

이 노트북은 **읽기 전용**입니다. 실습하려면 반드시 복사본을 만드세요!

```
📌 상단 메뉴 → 파일 → 드라이브에 사본 저장
```

복사본이 내 Google Drive에 저장되면, 그 파일에서 코드를 수정하고 실행할 수 있습니다.

In [ ]:
# ⚙️ 환경 설정
!pip install -q -U google-genai

from google import genai
from google.colab import userdata
import getpass
import pandas as pd

try:
    api_key = userdata.get('GEMINI_API_KEY')
    if isinstance(api_key, dict):
        api_key = str(list(api_key.values())[0])
    API_KEY = str(api_key).strip()
except Exception:
    API_KEY = getpass.getpass('🔑 Gemini API Key를 입력하세요: ')

client = genai.Client(api_key=API_KEY)
MODEL = 'gemini-2.5-flash'
print('✅ 준비 완료!')

In [ ]:
# ✏️ 학생 정보
academy_name = "달빛영어"
student_name = "김서연"
student_info = """초5 / B반 / 재원 8개월
이번 달 성적: 읽기 85, 듣기 90, 쓰기 70, 말하기 80
강점: 어휘력 우수, 수업 참여도 높음
약점: 문법 기초, 쓰기에서 시제 혼동
태도: 적극적이나 숙제 제출이 불규칙"""

## 🔬 실습 1: 피드백 4종 자동 생성

In [ ]:
feedback_types = [
    ('학생용 피드백', '학생이 스스로 읽고 동기부여 받을 수 있는 피드백. 칭찬 → 개선점 → 구체적 실천 방법. 이모지 활용. 200자 이내.'),
    ('학부모용 리포트', '학부모에게 보내는 공식 월간 리포트. 성적 데이터 + 강점 + 보완점 + 다음 달 계획. 전문적이고 신뢰감 있는 톤. 300자 이내.'),
    ('카카오톡 메시지', '학부모에게 보내는 간단한 카톡 메시지. 따뜻하고 친근한 톤. 이모지 포함. 100자 이내.'),
    ('4주 학습 전략', '학생의 약점을 보완하는 4주 맞춤 학습 전략. 주차별 목표 + 활동 + 확인방법. 표 형식.')
]

all_feedback = {}
for fb_name, fb_desc in feedback_types:
    prompt = f"""
너는 영어교육 전문가이자 학습 코치야.

아래 학생의 '{fb_name}'을 작성해줘.

학생: {student_name}
정보: {student_info}

요구사항: {fb_desc}
"""
    response = client.models.generate_content(model=MODEL, contents=prompt)
    all_feedback[fb_name] = response.text
    print(f'\n📝 {fb_name}')
    print('='*50)
    print(response.text)

## 🔬 실습 2: 배치 처리 — 5명 동시 피드백 생성 🔥

In [ ]:
# 📋 5명 학생 데이터 (DataFrame)
students_data = {
    '이름': ['김서연', '박지훈', '이하은', '최민준', '정수아'],
    '학년': ['초5', '초4', '초6', '초3', '초5'],
    '반': ['B', 'A', 'C', 'A', 'B'],
    '읽기': [85, 70, 92, 60, 78],
    '듣기': [90, 65, 88, 75, 82],
    '쓰기': [70, 80, 95, 55, 68],
    '말하기': [80, 75, 85, 70, 90],
    '강점': ['어휘력', '발음', '전 영역', '듣기 집중', '말하기 자신감'],
    '약점': ['문법', '읽기 속도', '없음', '쓰기 기초', '문법 정확도']
}

df = pd.DataFrame(students_data)
df['평균'] = df[['읽기', '듣기', '쓰기', '말하기']].mean(axis=1)
print('📊 학생 데이터:')
print(df.to_string(index=False))

In [ ]:
# 🔄 5명 전원 카톡 피드백 자동 생성
import time

batch_results = []

for _, row in df.iterrows():
    prompt = f"""
너는 {academy_name}의 영어 선생님이야.

아래 학생의 이번 달 학부모 카카오톡 메시지를 작성해줘.

학생: {row['이름']} ({row['학년']}/{row['반']}반)
성적: 읽기 {row['읽기']}, 듣기 {row['듣기']}, 쓰기 {row['쓰기']}, 말하기 {row['말하기']}
평균: {row['평균']:.0f}점
강점: {row['강점']}
약점: {row['약점']}

따뜻하고 친근한 톤. 이모지 포함. 100자 이내.
"""
    response = client.models.generate_content(model=MODEL, contents=prompt)
    batch_results.append({'이름': row['이름'], '메시지': response.text})
    print(f"✅ {row['이름']} 완료")
    time.sleep(1)

print('\n🎉 5명 전원 피드백 생성 완료!')
for r in batch_results:
    print(f"\n💬 {r['이름']}:")
    print(r['메시지'])

In [ ]:
# 💾 저장
output = ''
for r in batch_results:
    output += f"[{r['이름']}]\n{r['메시지']}\n\n"
with open('피드백_배치결과.txt', 'w', encoding='utf-8') as f:
    f.write(output)
from google.colab import files
files.download('피드백_배치결과.txt')
print('✅ 다운로드!')

## 🏆 4강 완료!
👉 [5강: 시험 대비 패키지](https://colab.research.google.com/github/Reasonofmoon/cedu-1/blob/main/notebooks/05_exam_prep.ipynb)